### 그래디언트 소실 & 가중치 초기화
parameter 수 적을수록.
-. 모델 구조는 알아서
-. 같은 구조상에서 더 나은 모델 ==> weight 0인게 많을수록 좋음
  ==> 그 parameter는 없는걸로. (선호)
-. 초기화 시에도 0에 가까운 값으로 초기화.

In [15]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import datetime
import tensorboard

#1
(x_train,y_train),(x_test,y_test) = mnist.load_data()

#2 정규화
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255.0
x_test /= 255.0

#3 one-hot
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# init = tf.keras.initializers.he_uniform()
init = tf.keras.initializers.RandomUniform(0.0,1.0)
# act = tf.keras.layers.sigmoid(alpha=0.3)
act = 'sigmoid'

In [13]:
n = 100
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(28,28)))
model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))
model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))
model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))
model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))
model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))
model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))

model.add(tf.keras.layers.Dense(units=10,activation='sigmoid',kernel_initializer=init))

model.summary()
opt = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_35 (Dense)            (None, 100)               78500     
                                                                 
 dense_36 (Dense)            (None, 100)               10100     
                                                                 
 dense_37 (Dense)            (None, 100)               10100     
                                                                 
 dense_38 (Dense)            (None, 100)               10100     
                                                                 
 dense_39 (Dense)            (None, 100)               10100     
                                                                 
 dense_40 (Dense)            (None, 100)              

In [18]:
import os
path = "/Users/user/Desktop/"
if not os.path.isdir(path):
  os.mkdir(path)
log_dir = path+"3203"

file_writer = tf.summary.create_file_writer(log_dir + "/gradient")
file_writer.set_as_default()

In [19]:
class GradientCallback(tf.keras.callbacks.Callback):
  def __init__(self,freq=10):
    self.freq = freq
    
  def on_epoch_end(self,epoch,logs):
    if epoch%self.freq != 0:
      return
    with tf.GradientTape() as tape:
      y_pred = model(x_train)
      loss = tf.keras.losses.binary_crossentropy(y_train,y_pred)
    grads = tape.gradient(loss,model.trainable_weights)
    for n in range(len(model.layers)):
      i2 = (n-1)*2
      i1 = i2+1
      
      bias_avg = tf.reduce_mean(tf.abs(grads[i1]))
      weight_avg = tf.reduce_mean(tf.abs(grads[i2]))
      
      tf.summary.scalar("layer_%d/avg/bias"%n,data=bias_avg,step=epoch)
      tf.summary.scalar("layer_%d/avg/weight"%n,data=weight_avg,step=epoch)
      
      tf.summary.histogram("layer_%d/hist/bias"%n,data=grads[i1],step=epoch)
      tf.summary.histogram("layer_%d/hist/weight"%n,data=grads[i2],step=epoch)
    def on_train_end(self,logs):
      tf.summary.flush()

In [20]:
callback1 = GradientCallback()
callback2 = tf.keras.callbacks.TensorBoard(
  log_dir=log_dir,
  histogram_freq=10
)

ret = model.fit(x_train,y_train,epochs=101,batch_size=200,
                validation_split=0.2,verbose=2,callbacks=[callback1,callback2])
train_loss,train_acc = model.evaluate(x_train,y_train,verbose=2)
test_loss,test_acc = model.evaluate(x_test,y_test,verbose=2)


Epoch 1/101
240/240 - 2s - loss: 2.4585 - accuracy: 0.1001 - val_loss: 2.4013 - val_accuracy: 0.1035 - 2s/epoch - 8ms/step
Epoch 2/101
240/240 - 1s - loss: 2.4436 - accuracy: 0.1032 - val_loss: 2.3542 - val_accuracy: 0.0997 - 684ms/epoch - 3ms/step
Epoch 3/101
240/240 - 1s - loss: 2.4382 - accuracy: 0.1044 - val_loss: 2.3794 - val_accuracy: 0.1060 - 688ms/epoch - 3ms/step
Epoch 4/101
240/240 - 1s - loss: 2.4426 - accuracy: 0.1010 - val_loss: 2.4047 - val_accuracy: 0.0998 - 699ms/epoch - 3ms/step
Epoch 5/101
240/240 - 1s - loss: 2.4425 - accuracy: 0.0993 - val_loss: 2.4191 - val_accuracy: 0.0975 - 676ms/epoch - 3ms/step
Epoch 6/101
240/240 - 1s - loss: 2.4423 - accuracy: 0.1031 - val_loss: 2.4092 - val_accuracy: 0.0975 - 661ms/epoch - 3ms/step
Epoch 7/101
240/240 - 1s - loss: 2.4408 - accuracy: 0.0996 - val_loss: 2.3370 - val_accuracy: 0.1035 - 657ms/epoch - 3ms/step
Epoch 8/101
240/240 - 1s - loss: 2.4420 - accuracy: 0.1004 - val_loss: 2.5712 - val_accuracy: 0.0997 - 700ms/epoch - 3ms/

In [17]:
%reload_ext tensorboard
%tensorboard --logdir {log_dir}

ERROR: Could not find `tensorboard`. Please ensure that your PATH
contains an executable `tensorboard` program, or explicitly specify
the path to a TensorBoard binary by setting the `TENSORBOARD_BINARY`
environment variable.

In [23]:
# 드롭아웃 비율 0.2로 학습시키기
act = tf.keras.layers.LeakyReLU(alpha=0.3)
init = 'he_uniform'
n = 100
dropout_rate = 0.2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=(32,32,3)))
model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))
model.add(tf.keras.layers.Dropout(rate=dropout_rate))

model.add(tf.keras.layers.Dense(units=n,activation=act,kernel_initializer=init))
model.add(tf.keras.layers.Dropout(rate=dropout_rate))

model.add(tf.keras.layers.Dense(units=10,activation='softmax'))
model.summary()

opt = tf.keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(optimizer=opt,loss='categorical_crossentropy',metrics=['accuracy'])

ret = model.fit(x_train,y_train,epochs=201,batch_size=400,
                validation_data = (x_test,y_test),verbose=0)

train_loss, train_acc = model.evaluate(x_train,y_train,verbose=2)
test_loss, test_acc = model.evaluate(x_test,y_test,verbose=2)


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 3072)              0         
                                                                 
 dense_48 (Dense)            (None, 100)               307300    
                                                                 
 dropout_4 (Dropout)         (None, 100)               0         
                                                                 
 dense_49 (Dense)            (None, 100)               10100     
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_50 (Dense)            (None, 10)                1010      
                                                                 
Total params: 318,410
Trainable params: 318,410
Non-tr

ValueError: in user code:

    File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1023, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\user\AppData\Roaming\Python\Python39\site-packages\keras\engine\input